In [1]:
import numpy as np
from sqlalchemy import create_engine
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.svm import SVC
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

In [2]:
filepath = '../Resources/BMI_Datasets/obese_low_risk.csv'
medical_df = pd.read_csv(filepath)
medical_df.head(10)

,ID#,Gender,Age,Hypertension,Heart_Disease,Ever_Married,Work_Type,Residence_Type,Avg_Glucose_Lvl,BMI,Smoker,Stroke
0,3,Male,80,0,1,Yes,Private,Rural,105.92,32.5,Never,1
1,4,Female,49,0,0,Yes,Private,Urban,171.23,34.4,Current,1
2,16,Female,50,1,0,Yes,Self-employed,Rural,167.41,30.9,Never,1
3,24,Male,82,0,1,Yes,Private,Rural,208.30,32.5,Unknown,1
4,34,Male,80,0,1,Yes,Self-employed,Urban,252.72,30.5,Former,1
5,37,Male,74,0,0,Yes,Private,Rural,219.72,33.7,Former,1
6,39,Male,58,0,0,No,Private,Rural,92.62,32.0,Unknown,1
7,46,Male,76,1,0,Yes,Private,Rural,243.58,33.6,Never,1
8,49,Male,81,0,0,Yes,Self-employed,Urban,99.33,33.7,Never,1
9,54,Female,77,1,0,Yes,Self-employed,Urban,124.13,31.4,Never,1


In [3]:
# Drop ID# column
medical_df.drop(columns=['ID#'], inplace=True)
medical_df.head()

,Gender,Age,Hypertension,Heart_Disease,Ever_Married,Work_Type,Residence_Type,Avg_Glucose_Lvl,BMI,Smoker,Stroke
0,Male,80,0,1,Yes,Private,Rural,105.92,32.5,Never,1
1,Female,49,0,0,Yes,Private,Urban,171.23,34.4,Current,1
2,Female,50,1,0,Yes,Self-employed,Rural,167.41,30.9,Never,1
3,Male,82,0,1,Yes,Private,Rural,208.30,32.5,Unknown,1
4,Male,80,0,1,Yes,Self-employed,Urban,252.72,30.5,Former,1


In [4]:
medical_df['Stroke'].value_counts()

0    944
1     56
Name: Stroke, dtype: int64

In [5]:
# Get a list of categorical columns
categorical_columns = medical_df.dtypes[medical_df.dtypes=='object'].index.tolist()
categorical_columns

['Gender', 'Ever_Married', 'Work_Type', 'Residence_Type', 'Smoker']

In [6]:
# Create OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit & transform OneHotEncoder using categorical columns
encode_df = pd.DataFrame(enc.fit_transform(medical_df[categorical_columns]))

# Add column names
encode_df.columns = enc.get_feature_names(categorical_columns)
print(encode_df.shape)
encode_df.head(10)

(1000, 15)


C:\Users\remem\anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Gender_Female,Gender_Male,Ever_Married_No,Ever_Married_Yes,Work_Type_Govt_job,Work_Type_Never_worked,Work_Type_Private,Work_Type_Self-employed,Work_Type_children,Residence_Type_Rural,Residence_Type_Urban,Smoker_Current,Smoker_Former,Smoker_Never,Smoker_Unknown
0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
2,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
5,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
6,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
7,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
8,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
9,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [7]:
# Get encode_df info
encode_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Gender_Female            1000 non-null   float64
 1   Gender_Male              1000 non-null   float64
 2   Ever_Married_No          1000 non-null   float64
 3   Ever_Married_Yes         1000 non-null   float64
 4   Work_Type_Govt_job       1000 non-null   float64
 5   Work_Type_Never_worked   1000 non-null   float64
 6   Work_Type_Private        1000 non-null   float64
 7   Work_Type_Self-employed  1000 non-null   float64
 8   Work_Type_children       1000 non-null   float64
 9   Residence_Type_Rural     1000 non-null   float64
 10  Residence_Type_Urban     1000 non-null   float64
 11  Smoker_Current           1000 non-null   float64
 12  Smoker_Former            1000 non-null   float64
 13  Smoker_Never             1000 non-null   float64
 14  Smoker_Unknown           

In [8]:
# Drop redundant columns
encode_df.drop(columns=['Gender_Female', 'Ever_Married_No', 'Residence_Type_Rural'], inplace=True)
encode_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Gender_Male              1000 non-null   float64
 1   Ever_Married_Yes         1000 non-null   float64
 2   Work_Type_Govt_job       1000 non-null   float64
 3   Work_Type_Never_worked   1000 non-null   float64
 4   Work_Type_Private        1000 non-null   float64
 5   Work_Type_Self-employed  1000 non-null   float64
 6   Work_Type_children       1000 non-null   float64
 7   Residence_Type_Urban     1000 non-null   float64
 8   Smoker_Current           1000 non-null   float64
 9   Smoker_Former            1000 non-null   float64
 10  Smoker_Never             1000 non-null   float64
 11  Smoker_Unknown           1000 non-null   float64
dtypes: float64(12)
memory usage: 93.9 KB


In [9]:
# Merge encoded df with medical df
medical_df = medical_df.merge(encode_df, left_index=True, right_index=True).drop(categorical_columns, axis=1)
print(medical_df.shape)
medical_df.head(10)

(1000, 18)


,Age,Hypertension,Heart_Disease,Avg_Glucose_Lvl,BMI,Stroke,Gender_Male,Ever_Married_Yes,Work_Type_Govt_job,Work_Type_Never_worked,Work_Type_Private,Work_Type_Self-employed,Work_Type_children,Residence_Type_Urban,Smoker_Current,Smoker_Former,Smoker_Never,Smoker_Unknown
0,80,0,1,105.92,32.5,1,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,49,0,0,171.23,34.4,1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
2,50,1,0,167.41,30.9,1,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,82,0,1,208.30,32.5,1,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,80,0,1,252.72,30.5,1,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
5,74,0,0,219.72,33.7,1,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6,58,0,0,92.62,32.0,1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7,76,1,0,243.58,33.6,1,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
8,81,0,0,99.33,33.7,1,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
9,77,1,0,124.13,31.4,1,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0


In [10]:
medical_df.isnull().sum()

Age                        0
Hypertension               0
Heart_Disease              0
Avg_Glucose_Lvl            0
BMI                        0
Stroke                     0
Gender_Male                0
Ever_Married_Yes           0
Work_Type_Govt_job         0
Work_Type_Never_worked     0
Work_Type_Private          0
Work_Type_Self-employed    0
Work_Type_children         0
Residence_Type_Urban       0
Smoker_Current             0
Smoker_Former              0
Smoker_Never               0
Smoker_Unknown             0
dtype: int64

In [11]:
# Create feature & target datasets
X = medical_df.drop(columns=['Stroke'])
y = medical_df['Stroke']
print(X.shape)
print(y.shape)

(1000, 17)
(1000,)


In [12]:
# Create empty arrays to store ML results
recalls = []
accuracies = []
precisions = []

# This will take a while. Feel free to adjust range to sample code. Feel free to review exported spreadsheets in github.
for i in range(1000):
    
    # Create training + testing data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
    
    # Oversample positive stroke cases in training set
    ros = RandomOverSampler()
    X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)
    
    # Scale feature data
    scaler = StandardScaler()
    scaler.fit(X_train_resampled)
    X_train_scaled = scaler.transform(X_train_resampled)
    X_test_scaled = scaler.transform(X_test)
    
    # Create SVC model
    svc_model = SVC(kernel='linear')
    svc_model = svc_model.fit(X_train_scaled, y_train_resampled)
    
    # Evaluate SVC model
    y_pred = svc_model.predict(X_test_scaled)
    
    # Add recall results to list
    recalls.append(recall_score(y_test, y_pred))
    
    # Add precision results to list
    precisions.append(precision_score(y_test, y_pred))
    
    # Add accuracy results to list
    accuracies.append(accuracy_score(y_test, y_pred))
    
    print("Current progress:", i)

Current progress: 0
Current progress: 1
Current progress: 2
Current progress: 3
Current progress: 4
Current progress: 5
Current progress: 6
Current progress: 7
Current progress: 8
Current progress: 9
Current progress: 10
Current progress: 11
Current progress: 12
Current progress: 13
Current progress: 14
Current progress: 15
Current progress: 16
Current progress: 17
Current progress: 18
Current progress: 19
Current progress: 20
Current progress: 21
Current progress: 22
Current progress: 23
Current progress: 24
Current progress: 25
Current progress: 26
Current progress: 27
Current progress: 28
Current progress: 29
Current progress: 30
Current progress: 31
Current progress: 32
Current progress: 33
Current progress: 34
Current progress: 35
Current progress: 36
Current progress: 37
Current progress: 38
Current progress: 39
Current progress: 40
Current progress: 41
Current progress: 42
Current progress: 43
Current progress: 44
Current progress: 45
Current progress: 46
Current progress: 47
Cu

Current progress: 378
Current progress: 379
Current progress: 380
Current progress: 381
Current progress: 382
Current progress: 383
Current progress: 384
Current progress: 385
Current progress: 386
Current progress: 387
Current progress: 388
Current progress: 389
Current progress: 390
Current progress: 391
Current progress: 392
Current progress: 393
Current progress: 394
Current progress: 395
Current progress: 396
Current progress: 397
Current progress: 398
Current progress: 399
Current progress: 400
Current progress: 401
Current progress: 402
Current progress: 403
Current progress: 404
Current progress: 405
Current progress: 406
Current progress: 407
Current progress: 408
Current progress: 409
Current progress: 410
Current progress: 411
Current progress: 412
Current progress: 413
Current progress: 414
Current progress: 415
Current progress: 416
Current progress: 417
Current progress: 418
Current progress: 419
Current progress: 420
Current progress: 421
Current progress: 422
Current pr

Current progress: 751
Current progress: 752
Current progress: 753
Current progress: 754
Current progress: 755
Current progress: 756
Current progress: 757
Current progress: 758
Current progress: 759
Current progress: 760
Current progress: 761
Current progress: 762
Current progress: 763
Current progress: 764
Current progress: 765
Current progress: 766
Current progress: 767
Current progress: 768
Current progress: 769
Current progress: 770
Current progress: 771
Current progress: 772
Current progress: 773
Current progress: 774
Current progress: 775
Current progress: 776
Current progress: 777
Current progress: 778
Current progress: 779
Current progress: 780
Current progress: 781
Current progress: 782
Current progress: 783
Current progress: 784
Current progress: 785
Current progress: 786
Current progress: 787
Current progress: 788
Current progress: 789
Current progress: 790
Current progress: 791
Current progress: 792
Current progress: 793
Current progress: 794
Current progress: 795
Current pr

In [13]:
precisions

[0.12162162162162163,
 0.109375,
 0.0847457627118644,
 0.09836065573770492,
 0.09433962264150944,
 0.1038961038961039,
 0.1044776119402985,
 0.04411764705882353,
 0.13414634146341464,
 0.0821917808219178,
 0.1232876712328767,
 0.11764705882352941,
 0.0963855421686747,
 0.12857142857142856,
 0.09259259259259259,
 0.12048192771084337,
 0.1323529411764706,
 0.12698412698412698,
 0.10144927536231885,
 0.13793103448275862,
 0.1016949152542373,
 0.10344827586206896,
 0.09090909090909091,
 0.11475409836065574,
 0.09722222222222222,
 0.14864864864864866,
 0.11864406779661017,
 0.10294117647058823,
 0.12962962962962962,
 0.11864406779661017,
 0.136986301369863,
 0.12307692307692308,
 0.13043478260869565,
 0.11594202898550725,
 0.11290322580645161,
 0.11764705882352941,
 0.11688311688311688,
 0.09230769230769231,
 0.07462686567164178,
 0.11475409836065574,
 0.10526315789473684,
 0.08695652173913043,
 0.14814814814814814,
 0.08928571428571429,
 0.10714285714285714,
 0.08823529411764706,
 0.1,
 0.

In [14]:
precisions_df = pd.DataFrame(precisions, columns=['Precision'])
print(precisions_df.shape)
precisions_df.head(10)

(1000, 1)


,Precision
0,0.121622
1,0.109375
2,0.084746
3,0.098361
4,0.094340
5,0.103896
6,0.104478
7,0.044118
8,0.134146
9,0.082192


In [15]:
accuracies

[0.665,
 0.695,
 0.7,
 0.7,
 0.73,
 0.64,
 0.68,
 0.635,
 0.645,
 0.64,
 0.67,
 0.75,
 0.61,
 0.685,
 0.725,
 0.63,
 0.695,
 0.71,
 0.67,
 0.735,
 0.71,
 0.715,
 0.675,
 0.71,
 0.655,
 0.685,
 0.72,
 0.675,
 0.745,
 0.72,
 0.68,
 0.7,
 0.775,
 0.68,
 0.705,
 0.685,
 0.65,
 0.68,
 0.66,
 0.71,
 0.72,
 0.755,
 0.755,
 0.715,
 0.615,
 0.665,
 0.705,
 0.71,
 0.65,
 0.745,
 0.69,
 0.635,
 0.725,
 0.71,
 0.73,
 0.645,
 0.69,
 0.6,
 0.64,
 0.7,
 0.73,
 0.72,
 0.695,
 0.765,
 0.615,
 0.685,
 0.69,
 0.705,
 0.695,
 0.68,
 0.74,
 0.65,
 0.675,
 0.66,
 0.695,
 0.7,
 0.68,
 0.74,
 0.745,
 0.7,
 0.695,
 0.7,
 0.68,
 0.665,
 0.725,
 0.745,
 0.665,
 0.725,
 0.72,
 0.705,
 0.68,
 0.675,
 0.67,
 0.7,
 0.695,
 0.67,
 0.66,
 0.6,
 0.69,
 0.68,
 0.63,
 0.695,
 0.71,
 0.67,
 0.705,
 0.655,
 0.68,
 0.705,
 0.7,
 0.71,
 0.745,
 0.675,
 0.7,
 0.71,
 0.695,
 0.68,
 0.675,
 0.665,
 0.68,
 0.66,
 0.665,
 0.7,
 0.715,
 0.625,
 0.695,
 0.68,
 0.68,
 0.63,
 0.7,
 0.69,
 0.695,
 0.73,
 0.68,
 0.73,
 0.695,
 0.76,
 0

In [16]:
accuracies_df = pd.DataFrame(accuracies, columns=['Accuracy'])
print(accuracies_df.shape)
accuracies_df.head(10)

(1000, 1)


,Accuracy
0,0.665
1,0.695
2,0.700
3,0.700
4,0.730
5,0.640
6,0.680
7,0.635
8,0.645
9,0.640


In [17]:
recalls

[0.8181818181818182,
 0.6363636363636364,
 0.45454545454545453,
 0.5454545454545454,
 0.45454545454545453,
 0.7272727272727273,
 0.6363636363636364,
 0.2727272727272727,
 1.0,
 0.5454545454545454,
 0.8181818181818182,
 0.5454545454545454,
 0.7272727272727273,
 0.8181818181818182,
 0.45454545454545453,
 0.9090909090909091,
 0.8181818181818182,
 0.7272727272727273,
 0.6363636363636364,
 0.7272727272727273,
 0.5454545454545454,
 0.5454545454545454,
 0.5454545454545454,
 0.6363636363636364,
 0.6363636363636364,
 1.0,
 0.6363636363636364,
 0.6363636363636364,
 0.6363636363636364,
 0.6363636363636364,
 0.9090909090909091,
 0.7272727272727273,
 0.5454545454545454,
 0.7272727272727273,
 0.6363636363636364,
 0.7272727272727273,
 0.8181818181818182,
 0.5454545454545454,
 0.45454545454545453,
 0.6363636363636364,
 0.5454545454545454,
 0.36363636363636365,
 0.7272727272727273,
 0.45454545454545453,
 0.8181818181818182,
 0.5454545454545454,
 0.5454545454545454,
 0.6363636363636364,
 0.6363636363636

In [18]:
recalls_df = pd.DataFrame(recalls, columns=['Recall'])
print(recalls_df.shape)
recalls_df.head(10)

(1000, 1)


,Recall
0,0.818182
1,0.636364
2,0.454545
3,0.545455
4,0.454545
5,0.727273
6,0.636364
7,0.272727
8,1.000000
9,0.545455


In [19]:
results_df = accuracies_df.merge(precisions_df, left_index=True, right_index=True)
print(results_df.shape)
results_df.head(10)

(1000, 2)


,Accuracy,Precision
0,0.665,0.121622
1,0.695,0.109375
2,0.700,0.084746
3,0.700,0.098361
4,0.730,0.094340
5,0.640,0.103896
6,0.680,0.104478
7,0.635,0.044118
8,0.645,0.134146
9,0.640,0.082192


In [20]:
results_df = results_df.merge(recalls_df, left_index=True, right_index=True)
print(results_df.shape)
results_df.head(10)

(1000, 3)


,Accuracy,Precision,Recall
0,0.665,0.121622,0.818182
1,0.695,0.109375,0.636364
2,0.700,0.084746,0.454545
3,0.700,0.098361,0.545455
4,0.730,0.094340,0.454545
5,0.640,0.103896,0.727273
6,0.680,0.104478,0.636364
7,0.635,0.044118,0.272727
8,0.645,0.134146,1.000000
9,0.640,0.082192,0.545455


In [21]:
print("Recall mean: " + str(results_df['Recall'].mean()))
print("Recall mode: " + str(results_df['Recall'].mode()))
print("Recall median: " + str(results_df['Recall'].median()))
print("Recall max: " + str(results_df['Recall'].max()))
print("Recall min: " + str(results_df['Recall'].min()))
print("Recall standard deviation: " + str(results_df['Recall'].std()))

Recall mean: 0.659000000000002
Recall mode: 0    0.727273
dtype: float64
Recall median: 0.6363636363636364
Recall max: 1.0
Recall min: 0.2727272727272727
Recall standard deviation: 0.1428015575784471


In [22]:
print("Accuracy mean: " + str(results_df['Accuracy'].mean()))
print("Accuracy mode: " + str(results_df['Accuracy'].mode()))
print("Accuracy median: " + str(results_df['Accuracy'].median()))
print("Accuracy max: " + str(results_df['Accuracy'].max()))
print("Accuracy min: " + str(results_df['Accuracy'].min()))
print("Accuracy standard deviation: " + str(results_df['Accuracy'].std()))

Accuracy mean: 0.689195000000001
Accuracy mode: 0    0.69
dtype: float64
Accuracy median: 0.69
Accuracy max: 0.8
Accuracy min: 0.575
Accuracy standard deviation: 0.03371666930153078


In [23]:
print("Precision mean: " + str(results_df['Precision'].mean()))
print("Precision mode: " + str(results_df['Precision'].mode()))
print("Precision median: " + str(results_df['Precision'].median()))
print("Precision max: " + str(results_df['Precision'].max()))
print("Precision min: " + str(results_df['Precision'].min()))
print("Precision standard deviation: " + str(results_df['Precision'].std()))

Precision mean: 0.11053422366908545
Precision mode: 0    0.1
dtype: float64
Precision median: 0.11043360433604335
Precision max: 0.1774193548387097
Precision min: 0.04411764705882353
Precision standard deviation: 0.021242149043329883


In [24]:
results_df.isnull().sum().sum()

0

In [25]:
len(results_df)

1000

In [ ]:
filepath = ('ML_result_files/SVC_ObeseLowRisk_SimpleImputer(mean)_RandomOverSampler.csv')
results_df.to_csv(filepath, index=False)